In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

from pyspark.ml import Pipeline
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.ml.evaluation import BinaryClassificationEvaluator, RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.regression import DecisionTreeRegressor

In [0]:
IS_SPARK_SUBMIT_CLI = True

if IS_SPARK_SUBMIT_CLI:
    sc = SparkContext.getOrCreate()
    spark = SparkSession(sc)

#LOAD DATA  -- FareSample.csv and TripSample.csv

In [0]:
file_location = "/FileStore/tables/fareSample.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df1 = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)
  
file_location = "/FileStore/tables/tripSample.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df2 = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

In [0]:
temp_table_name1 = "fareSample_csv"

df1.createOrReplaceTempView(temp_table_name1)

In [0]:
temp_table_name2 = "tripSample_csv"

df2.createOrReplaceTempView(temp_table_name2)

In [0]:
if IS_SPARK_SUBMIT_CLI:
    fareSample = spark.read.csv('/FileStore/tables/fareSample.csv', inferSchema=True, header=True)
else:
    fareSample = sqlContext.sql("select * from fareSample_csv")

In [0]:
if IS_SPARK_SUBMIT_CLI:
    tripSample = spark.read.csv('/FileStore/tables/tripSample.csv', inferSchema=True, header=True)
else:
    tripSample = sqlContext.sql("select * from tripSample_csv")

#Combine two datasets

In [0]:
data=fareSample.join(tripSample,  "medallion","inner")

data.show(5)

+--------------------+--------------------+----------+-------------------+-------------+------------+----------+--------+-----------+-------------+-------------+--------------------+---------+---------+------------------+-------------------+-------------------+---------------+-----------------+-------------+----------------+---------------+-----------------+----------------+
 medallion| hack_license| vendor_id| pickup_datetime| payment_type| fare_amount| surcharge| mta_tax| tip_amount| tolls_amount| total_amount| hack_license|vendor_id|rate_code|store_and_fwd_flag| pickup_datetime| dropoff_datetime|passenger_count|trip_time_in_secs|trip_distance|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|
+--------------------+--------------------+----------+-------------------+-------------+------------+----------+--------+-----------+-------------+-------------+--------------------+---------+---------+------------------+-------------------+-------------------+---------------+-----------------+-------------+----------------+---------------+-----------------+----------------+
89D227B655E5C82AE...|BA96DE419E711691B...| CMT|2013-01-01 15:11:48| CSH| 6.5| 0.0| 0.5| 0.0| 0.0| 7.0|BA96DE419E711691B...| CMT| 1| N|2013-01-01 15:11:48|2013-01-01 15:18:10| 4| 382| 1.0| -73.978165| 40.757977| -73.989838| 40.751171|
0BD7C8F5BA12B88E0...|9FD8F69F0804BDB55...| CMT|2013-01-06 00:18:35| CSH| 6.0| 0.5| 0.5| 0.0| 0.0| 7.0|9FD8F69F0804BDB55...| CMT| 1| N|2013-01-05 18:49:41|2013-01-05 18:54:23| 1| 282| 1.1| -74.004707| 40.73777| -74.009834| 40.726002|
0BD7C8F5BA12B88E0...|9FD8F69F0804BDB55...| CMT|2013-01-06 00:18:35| CSH| 6.0| 0.5| 0.5| 0.0| 0.0| 7.0|9FD8F69F0804BDB55...| CMT| 1| N|2013-01-06 00:18:35|2013-01-06 00:22:54| 1| 259| 1.5| -74.006683| 40.731781| -73.994499| 40.75066|
0BD7C8F5BA12B88E0...|9FD8F69F0804BDB55...| CMT|2013-01-05 18:49:41| CSH| 5.5| 1.0| 0.5| 0.0| 0.0| 7.0|9FD8F69F0804BDB55...| CMT| 1| N|2013-01-05 18:49:41|2013-01-05 18:54:23| 1| 282| 1.1| -74.004707| 40.73777| -74.009834| 40.726002|
0BD7C8F5BA12B88E0...|9FD8F69F0804BDB55...| CMT|2013-01-05 18:49:41| CSH| 5.5| 1.0| 0.5| 0.0| 0.0| 7.0|9FD8F69F0804BDB55...| CMT| 1| N|2013-01-06 00:18:35|2013-01-06 00:22:54| 1| 259| 1.5| -74.006683| 40.731781| -73.994499| 40.75066|
+--------------------+--------------------+----------+-------------------+-------------+------------+----------+--------+-----------+-------------+-------------+--------------------+---------+---------+------------------+-------------------+-------------------+---------------+-----------------+-------------+----------------+---------------+-----------------+----------------+
only showing top 5 rows

#Select Data

In [0]:
data1 = data.select('passenger_count', 'trip_time_in_secs', 'trip_distance', col(' fare_amount').alias('label'))

In [0]:
data1.show(5)

+---------------+-----------------+-------------+-----+
passenger_count|trip_time_in_secs|trip_distance|label|
+---------------+-----------------+-------------+-----+
 4| 382| 1.0| 6.5|
 1| 259| 1.5| 5.5|
 1| 259| 1.5| 6.0|
 1| 282| 1.1| 5.5|
 1| 282| 1.1| 6.0|
+---------------+-----------------+-------------+-----+
only showing top 5 rows

#Setup Train and Test datasets

In [0]:
splits = data1.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1].withColumnRenamed("label", "trueLabel")

#Boosted Decision Tree Regression

In [0]:
assembler = VectorAssembler(inputCols = ['passenger_count', 'trip_time_in_secs', 'trip_distance'], outputCol="features")
gbt = GBTRegressor(labelCol="label")

In [0]:
paramGrid = ParamGridBuilder()\
  .addGrid(gbt.maxDepth, [2, 3])\
  .addGrid(gbt.maxIter, [10, 20])\
  .build()
  
evaluator = RegressionEvaluator(metricName="rmse", labelCol=gbt.getLabelCol(), predictionCol=gbt.getPredictionCol())

cv = CrossValidator(estimator=gbt, evaluator=evaluator, estimatorParamMaps=paramGrid)

In [0]:
pipeline = Pipeline(stages=[assembler, cv])
pipelineModel = pipeline.fit(train)

/databricks/spark/python/pyspark/ml/util.py:800: UserWarning: Cannot find mlflow module. To enable MLflow logging, install mlflow from PyPI.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)

In [0]:
predictions = pipelineModel.transform(test)

In [0]:
predicted = predictions.select("features", "prediction", "trueLabel")
predicted.show(10)

+-----------------+------------------+---------+
 features| prediction|trueLabel|
+-----------------+------------------+---------+
[0.0,1020.0,5.21]| 9.755882501085065| 6.5|
[0.0,1020.0,5.21]| 9.755882501085065| 7.0|
[0.0,1020.0,5.21]| 9.755882501085065| 28.0|
[0.0,1020.0,5.21]| 9.755882501085065| 50.0|
 [1.0,0.0,0.0]|12.130604867394199| 2.5|
 [1.0,0.0,0.0]|12.130604867394199| 2.5|
 [1.0,0.0,0.0]|12.130604867394199| 2.5|
 [1.0,0.0,0.0]|12.130604867394199| 2.5|
 [1.0,0.0,0.0]|12.130604867394199| 2.5|
 [1.0,0.0,0.0]|12.130604867394199| 2.5|
+-----------------+------------------+---------+
only showing top 10 rows

In [0]:
predicted.createOrReplaceTempView("regressionPredictions")

In [0]:
dataPred = spark.sql("SELECT trueLabel, prediction FROM regressionPredictions")

dataPred.show(5)

+---------+------------------+
trueLabel| prediction|
+---------+------------------+
 6.5| 9.755882501085065|
 7.0| 9.755882501085065|
 28.0| 9.755882501085065|
 50.0| 9.755882501085065|
 2.5|12.130604867394199|
+---------+------------------+
only showing top 5 rows

#RMSE for GBT-Regression

In [0]:
evaluator  = RegressionEvaluator(labelCol="trueLabel", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print( "Root Mean Square Error (RMSE) for GBT Regression :", rmse)

Root Mean Square Error (RMSE) for GBT Regression : 9.098194939712792

#Linear Regression

In [0]:
assembler = VectorAssembler(inputCols = ['passenger_count', 'trip_time_in_secs', 'trip_distance'], outputCol="features")
lr = LinearRegression(labelCol="label",featuresCol="features", maxIter=10, regParam=0.3)
pipeline1 = Pipeline(stages=[assembler, lr])

In [0]:
paramGrid1 = ParamGridBuilder().addGrid(lr.regParam, [0.3, 0.01]).addGrid(lr.maxIter, [10, 5]).build()
trainval = TrainValidationSplit(estimator=pipeline1, evaluator=RegressionEvaluator(), estimatorParamMaps=paramGrid1, trainRatio=0.8)

In [0]:
pipelineModel = trainval.fit(train)

In [0]:
predictions = pipelineModel.transform(test)

In [0]:
predicted = predictions.select("features", "prediction", "trueLabel")
predicted.show(10)

+-----------------+------------------+---------+
 features| prediction|trueLabel|
+-----------------+------------------+---------+
[0.0,1020.0,5.21]|12.011785584117332| 6.5|
[0.0,1020.0,5.21]|12.011785584117332| 7.0|
[0.0,1020.0,5.21]|12.011785584117332| 28.0|
[0.0,1020.0,5.21]|12.011785584117332| 50.0|
 [1.0,0.0,0.0]|10.383540701432295| 2.5|
 [1.0,0.0,0.0]|10.383540701432295| 2.5|
 [1.0,0.0,0.0]|10.383540701432295| 2.5|
 [1.0,0.0,0.0]|10.383540701432295| 2.5|
 [1.0,0.0,0.0]|10.383540701432295| 2.5|
 [1.0,0.0,0.0]|10.383540701432295| 2.5|
+-----------------+------------------+---------+
only showing top 10 rows

In [0]:
predicted.createOrReplaceTempView("regressionPredictions")

In [0]:
dataPred = spark.sql("SELECT trueLabel, prediction FROM regressionPredictions")

dataPred.show(5)

+---------+------------------+
trueLabel| prediction|
+---------+------------------+
 6.5|12.011785584117332|
 7.0|12.011785584117332|
 28.0|12.011785584117332|
 50.0|12.011785584117332|
 2.5|10.383540701432295|
+---------+------------------+
only showing top 5 rows

#RMSE for Linear Regression

In [0]:
evaluator  = RegressionEvaluator(labelCol="trueLabel", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print( "Root Mean Square Error (RMSE) for Linear Regression :", rmse)

Root Mean Square Error (RMSE) for Linear Regression : 9.101831968270446

#Decision Forest Regression

In [0]:
assembler = VectorAssembler(inputCols = ['passenger_count', 'trip_time_in_secs', 'trip_distance'], outputCol="features")
dt = DecisionTreeRegressor(labelCol="label",featuresCol="features")

In [0]:
paramGrid2 = ParamGridBuilder()\
  .addGrid(dt.maxDepth, [2,3])\
  .addGrid(dt.maxBins, [10,20])\
  .build()

In [0]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol=dt.getLabelCol(), predictionCol=dt.getPredictionCol())

dtcv = CrossValidator(estimator = dt, estimatorParamMaps = paramGrid2, evaluator = evaluator, numFolds=2)

In [0]:
pipeline2 = Pipeline(stages=[assembler, dtcv])
pipelineModel = pipeline2.fit(train)

In [0]:
predictions = pipelineModel.transform(test)

In [0]:
predicted = predictions.select("features","prediction","truelabel")
predicted.show(10)

+-----------------+------------------+---------+
 features| prediction|truelabel|
+-----------------+------------------+---------+
[0.0,1020.0,5.21]|11.504112878301596| 6.5|
[0.0,1020.0,5.21]|11.504112878301596| 7.0|
[0.0,1020.0,5.21]|11.504112878301596| 28.0|
[0.0,1020.0,5.21]|11.504112878301596| 50.0|
 [1.0,0.0,0.0]|10.782542570855586| 2.5|
 [1.0,0.0,0.0]|10.782542570855586| 2.5|
 [1.0,0.0,0.0]|10.782542570855586| 2.5|
 [1.0,0.0,0.0]|10.782542570855586| 2.5|
 [1.0,0.0,0.0]|10.782542570855586| 2.5|
 [1.0,0.0,0.0]|10.782542570855586| 2.5|
+-----------------+------------------+---------+
only showing top 10 rows

In [0]:
predicted.createOrReplaceTempView("regressionPredictions")

In [0]:
dataPred = spark.sql("SELECT trueLabel, prediction FROM regressionPredictions")

dataPred.show(5)

+---------+------------------+
trueLabel| prediction|
+---------+------------------+
 6.5|11.504112878301596|
 7.0|11.504112878301596|
 28.0|11.504112878301596|
 50.0|11.504112878301596|
 2.5|10.782542570855586|
+---------+------------------+
only showing top 5 rows

#RMSE for Decision Forest Regression

In [0]:
evaluator  = RegressionEvaluator(labelCol="trueLabel", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print( "Root Mean Square Error (RMSE) for Decision Forest Regression :", rmse)

Root Mean Square Error (RMSE) for Decision Forest Regression : 9.102872234972422